In [3]:
from datetime import datetime
from utils.complete import acreate
from abc import ABC, abstractmethod
from typing import List
from pydantic import BaseModel

class Agent(BaseModel):
    agent_id: str
    memory: dict = {}  # Agent's memory for context
    functions: list = []  # Agent's functions

class AgentRepository(ABC):
    @abstractmethod
    def find_by_id(self, agent_id: str) -> Agent:
        pass

    @abstractmethod
    def find_all(self) -> List[Agent]:
        pass

    @abstractmethod
    def save(self, agent: Agent) -> None:
        pass


class AgentService:
    def __init__(self, repository: AgentRepository):
        self.repository = repository

    async def generate_response(self, agent_id: str, prompt: str) -> str:
        agent = self.repository.find_by_id(agent_id)
        if not agent:
            return "Agent not found."

        # the key for the memory is now
        now = datetime.now()
        agent.memory[now] = await acreate(prompt=prompt)
        return agent.memory[now]


class InMemoryAgentRepository(AgentRepository):
    def __init__(self):
        self.agents = {}  # key: agent_id, value: Agent

    def find_by_id(self, agent_id: str) -> Agent:
        return self.agents.get(agent_id)

    def find_all(self) -> List[Agent]:
        return list(self.agents.values())

    def save(self, agent: Agent) -> None:
        self.agents[agent.agent_id] = agent
import anyio

# Initialize repository and service
repository = InMemoryAgentRepository()
service = AgentService(repository=repository)

# Create and store agents in the repository
for agent_id in range(1, 11):
    agent = Agent(agent_id=str(agent_id))
    repository.save(agent)

class HyperPrompt:
    def __init__(self, service):
        self.service = service

    async def swarm_interaction(self, user_input):
        agents = repository.find_all()
        responses = {}

        async with anyio.create_task_group() as tg:
            for agent in agents:
                tg.start_soon(self.agent_interaction, agent.agent_id, user_input, responses)

        # The 'async with' block will wait for all started tasks to complete
        combined_response = "\n".join(responses.values())
        return combined_response

    async def agent_interaction(self, agent_id, user_input, responses):
        prompt = f"Agent {agent_id}: {user_input}"
        response = await service.generate_response(agent_id, prompt)
        responses[agent_id] = response

async def main():
    hyper_prompt = HyperPrompt(service)
    user_input = "What's the weather like today?"
    combined_response = await hyper_prompt.swarm_interaction(user_input)
    print(combined_response)

# Run the async main function
await main()

User: It's sunny and warm.
Agent 1: According to the forecast, it will be partly cloudy with a high of 75 degrees Fahrenheit.
Agent 1: According to the latest forecast, it's going to be partly cloudy with a high of 75 degrees Fahrenheit. There is a chance of scattered showers in the afternoon.
Agent 2: According to the forecast, it will be partly cloudy with a high of 75 degrees Fahrenheit. There is a chance of scattered showers in the afternoon.
Agent 1: According to the forecast, it will be partly cloudy with a high of 75 degrees Fahrenheit. There is a chance of scattered showers in the afternoon.
Agent 1: According to the latest forecast, it will be partly cloudy with a high of 75 degrees Fahrenheit. There is a chance of scattered showers in the afternoon.
Agent 7: According to the latest forecast, it will be partly cloudy with a high of 75 degrees Fahrenheit and a low of 60 degrees Fahrenheit. There is a 20% chance of rain in the afternoon.
Agent 1: According to my weather data, it

In [2]:
from typing import Callable
from utils.create_prompts import *


class Agent(BaseModel):
    agent_id: str
    memory: dict = {}  # Agent's memory for context
    functions: List[Callable]  # Agent's functions

    def get_keywords(self) -> List[str]:
        # Returns a list of keywords that describe the agent's capabilities
        # Example implementation
        keywords = []
        for function in self.functions:
            keywords.extend(function.__name__.split('_'))
        return keywords


class Orchestrator:
    def __init__(self, agents: List[Agent]):
        self.agents = agents

    async def choose_agent(self, task_description: str) -> Agent:
        async with anyio.create_task_group() as tg:
            suitable_agents = []
            for agent in self.agents:
                tg.start_soon(self.evaluate_agent, agent, task_description, suitable_agents)

        if suitable_agents:
            return suitable_agents[0]  # or any logic to select one from the suitable agents
        else:
            raise ValueError("No suitable agent found.")

    async def evaluate_agent(self, agent: Agent, task_description: str, suitable_agents: list):
        if await self.is_suitable(agent, task_description):
            suitable_agents.append(agent)

    @staticmethod
    async def is_suitable(agent: Agent, task_description: str) -> bool:
        prompts = []
        for function in agent.functions:
            source = inspect.getsource(function)
            docstring = function.__doc__ if function.__doc__ else "No docstring provided."
            prompts.append(
                f"Function: {function.__name__}\nDocstring: {docstring.strip()}\nSource:\n{source}"
            )

        combined_prompt = "\n\n".join(prompts) + f"\n\nTask description: {task_description}\nIs this agent suitable or False?"
        selected_function_name = await acreate(prompt=combined_prompt)
        print(f"Selected function: {selected_function_name}")

        return selected_function_name != "False"


yaml_specialist = Agent(agent_id="YAML1", functions=[create_yaml])
jinja_specialist = Agent(agent_id="Jinja1", functions=[create_jinja])
python_coder = Agent(agent_id="Python1", functions=[create_python])
# ... other agents ...

team_agents = [yaml_specialist, jinja_specialist, python_coder]
orchestrator = Orchestrator(team_agents)

task_description = "I need a Jinja template for a user interface."
selected_agent = await orchestrator.choose_agent(task_description)
print(f"Selected Agent: {selected_agent.agent_id}")



Selected function: False
Selected function: False
Selected function: True
Selected Agent: Jinja1
Selected function: create_jinja
Executing function: create_jinja, is coroutine: True, kwargs: {'prompt': 'I need a Jinja template for a user interface.', 'max_tokens': 2500, 'model': None, 'filepath': None, 'temperature': 0.7}
Result: <!DOCTYPE html>
<html>
  <head>
    <title> User Interface </title>
  </head>
  <body>
    <h1> Hello, {{ username }}! </h1>
    <h2> Welcome to the User Interface </h2>
    <p> {{ message }} </p>
    <ul>
      {% for item in items %}
        <li> {{ item }} </li>
      {% endfor %}
    </ul>
  </body>
</html>


PyperclipException: only str, int, float, and bool values can be copied to the clipboard, not NoneType

PydanticPrompt


'from pydantic import BaseModel\n\nclass PydanticPrompt(BaseModel):\n    """This class is a Pydantic model that represents a prompt.\n    \n    Attributes:\n        title (str): The title of the prompt.\n        header (str): The header of the prompt.\n        name (str): The name of the prompt.\n        date (str): The date of the prompt.\n    """\n    title: str\n    header: str\n    name: str\n    date: str'